In [1]:
from Animations import MoveAnimation
from moviepy.editor import ImageSequenceClip, ImageClip, CompositeVideoClip, VideoFileClip, concatenate_videoclips

import os

In [1]:

def remove_suffix(folder_path, suffix):
    files = os.listdir(folder_path)
    for file in files:
        if os.path.isfile(os.path.join(folder_path, file)):
            file_name, file_ext = os.path.splitext(file)
            if file_name.endswith(suffix):
                new_file_name = file_name[:-len(suffix)] + file_ext
                os.rename(os.path.join(folder_path, file), os.path.join(folder_path, new_file_name))

# Example usage
frame_path = 'images\\frames'
suffix = '_delay'
remove_suffix(frame_path, suffix)

## Having fun with videos editing

### frames to video

In [2]:

images_list = [os.path.join('images\\frames', frame) for frame in os.listdir('images\\frames')] # your images list
clip = ImageSequenceClip(images_list, fps=24)  # fps: frames per second
clip.write_videofile("output_video.mp4")

Moviepy - Building video output_video.mp4.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4


In [3]:
def create_video_from_frames(frame_path, output_video_path, fps):
    images_list = [os.path.join(frame_path, frame) for frame in os.listdir(frame_path)]
    clip = ImageSequenceClip(images_list, fps=fps)
    clip.write_videofile(output_video_path)

create_video_from_frames('images\\frames', 'mecquidanse.mp4', 24)

Moviepy - Building video mecquidanse.mp4.
Moviepy - Writing video mecquidanse.mp4



Moviepy - Done !
Moviepy - video ready mecquidanse.mp4


### add picture overlay 

In [4]:

images_list = [os.path.join('images\\frames', frame) for frame in os.listdir('images\\frames')]
clip = ImageSequenceClip(images_list, fps=20)

# Load the image to be added
image_clip = ImageClip("jai_peur.png").set_start(0).set_duration(3)

# Resize and position the image in the middle of the screen
image_clip = image_clip.resize(width=clip.w/2, height=clip.h/2)
x = (clip.w - image_clip.w) / 2
y = (clip.h - image_clip.h) / 2
image_clip = image_clip.set_pos((x, y))

# Overlay the image on the video
final_clip = CompositeVideoClip([clip, image_clip])

# Save the video
final_clip.write_videofile("my_edited_video.mp4")

Moviepy - Building video my_edited_video.mp4.
Moviepy - Writing video my_edited_video.mp4



Moviepy - Done !
Moviepy - video ready my_edited_video.mp4


In [5]:
def add_image_to_video(input_video_path, output_video_path, image_path, start_time = 0, duration = -1, x_pos = 0, y_pos=0, width=-1, height=-1):
    clip = VideoFileClip(input_video_path)

    if(duration == -1):
        duration = clip.duration - start_time
    if(width == -1):
        width = clip.w
    if(height == -1):
        height = clip.h
    
    

    # Split the video into three parts
    clip_before = clip.subclip(0, start_time)
    clip_middle = clip.subclip(start_time, start_time + duration)
    clip_after = clip.subclip(start_time + duration, clip.duration)

    image_clip = ImageClip(image_path)
    image_clip = image_clip.resize(width=width, height=height)
    image_clip = image_clip.set_pos((x_pos, y_pos))

    image_clip = image_clip.set_start(0).set_duration(clip_middle.duration)

    final_clip_middle = CompositeVideoClip([clip_middle, image_clip])

    final_clip = concatenate_videoclips([clip_before, final_clip_middle, clip_after])

    # Save the video
    final_clip.write_videofile(output_video_path)

In [6]:
add_image_to_video('mecquidanse.mp4', 'mecquidanse_with_image.mp4', 'jai_peur.png', 3, 2, 0, 0, 160, 90)

Moviepy - Building video mecquidanse_with_image.mp4.
Moviepy - Writing video mecquidanse_with_image.mp4



Moviepy - Done !
Moviepy - video ready mecquidanse_with_image.mp4


### animate Image
#### Move

In [9]:
danse = VideoFileClip('mecquidanse.mp4')
print(f'fps: {danse.fps}, duration: {danse.duration}, size: {danse.size}')

fps: 24.0, duration: 5.25, size: [260, 200]


In [3]:
from Animations import MoveAnimation

def animate_image(input_video_path, output_video_path, image_path, anim: MoveAnimation):
    clip = VideoFileClip(input_video_path)

    
    
    if(anim.duration == -1):
        anim.duration = clip.duration - anim.start_time
    if(anim.start_size == (-1,-1)):
        anim.start_size = (clip.w, clip.h)
    if(anim.end_size == (-1,-1)):
        anim.end_size = (clip.w, clip.h)
    
    

    # Split the video into three parts
    clip_before = clip.subclip(0, anim.start_time)
    clip_middle = clip.subclip(anim.start_time, anim.start_time + anim.duration)
    clip_after = clip.subclip(anim.start_time + anim.duration, clip.duration)

    image_clip = ImageClip(image_path)
    image_clip.fps = clip.fps
    if(anim.start_size == (0,0)):
        image_clip = image_clip.resize((1,1))
    else: 
        image_clip = image_clip.resize(anim.start_size)
    image_clip = image_clip.set_pos(anim.start_pos)

    
    image_clip = image_clip.set_start(0).set_duration(clip_middle.duration)

    move_func = lambda t: anim.get_pos(t)
    image_clip = image_clip.set_position(move_func)
    
    size_func = lambda t: anim.get_size(t)
    image_clip = image_clip.resize(size_func) # if size_func != (0,0) else None
    
    final_clip_middle = CompositeVideoClip([clip_middle, image_clip])
    final_clip = concatenate_videoclips([clip_before, final_clip_middle, clip_after])

    # Save the video
    final_clip.write_videofile(output_video_path)

In [7]:
moveAnim = MoveAnimation(1, 2, (0, 0), (160, 90), end_size=(10, 10))

animate_image('mecquidanse.mp4', 'mecquidanse_with_image.mp4', 'jai_peur.png', moveAnim)

Moviepy - Building video mecquidanse_with_image.mp4.
Moviepy - Writing video mecquidanse_with_image.mp4



Moviepy - Done !
Moviepy - video ready mecquidanse_with_image.mp4


## Find Images from text

## Find moment from text

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)
a=torch.cuda.FloatTensor()

True
2.3.1+cu121


C:\Users\lorra\AppData\Local\Temp\ipykernel_10592\2954205286.py:4: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:80.)
  a=torch.cuda.FloatTensor()


In [4]:
import os
print(os.environ['CUDA_HOME'])

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5


In [5]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="microsoft/Phi-3-vision-128k-instruct", trust_remote_code=True)
pipe(messages)

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- configuration_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- image_embedding_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- modeling_phi3_v.py
- image_embedding_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.